In [3]:
import pandas as pd
articles = pd.read_csv("articles.CDKN2A.csv")
authors = pd.read_csv("authors.CDKN2A.csv")
paper = pd.read_csv("paper_counts.csv")

In [4]:
print(articles)

          PMID                                              Title  \
0     10551774  Transfection of an inducible p16/CDKN2A constr...   
1     10595918  Malignant transformation of neurofibromas in n...   
2     10620111  Genotype/phenotype and penetrance studies in m...   
3     10630172  The genetics of hereditary melanoma and nevi. ...   
4     10632344  Analysis of oncogene and tumor suppressor gene...   
...        ...                                                ...   
3409  38067238  Validation of a Gene Expression Approach for t...   
3410  38070141  Has-miR-300-GADD45B promotes melanoma growth v...   
3411  38079020  "De novo replication repair deficient glioblas...   
3412  38091828  Dry and wet experiments reveal diagnostic clus...   
3413  38095311  Compliant substrates mitigate the senescence a...   

                                               Abstract       ISSN  \
0     Recent studies have shown that methylation of ...  0888-8809   
1     Patients with neurofibrom